<a href="https://colab.research.google.com/github/dubeyabhi07/big-data-spark/blob/master/src/main/pyspark/stocks/stockAnalysisSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

     |████████████████████████████████| 218.4MB 57kB/s 
     |████████████████████████████████| 204kB 47.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814406 sha256=89f0275fec56e2158736b0e8de820e5ee62102cc29b8323a5a016d5d6afde018
  Stored in directory: /root/.cache/pip/wheels/1e/5e/6a/17e906c94ec7246f260330a66e44a06a0809033ba2738a74a8
Successfully built pyspark


In [4]:
stocksToBeLoaded = {"HDFC" : "DLF.NS.csv",
                    "ICICI" : "ICICIBANK.NS.csv",
                    "cipla" : "CIPLA.NS.csv",
                    "HUL" :"HINDUNILVR.NS.csv",
                    "sunpharma" : "SUNPHARMA.NS.csv",
                    "DLF":"DLF.NS.csv",
                    "indiaBulls":"IBREALEST.NS.csv",                    
                    "reliance":"RELIANCE.NS.csv",
                    "ONGC":"ONGC.NS.csv",
                    "tataMotor":"TATAMOTORS.NS.csv",
                    "hero":"HEROMOTOCO.NS.csv",
                    "symphony":"SYMPHONY.NS.csv",
                    "whirlpool":"WHIRLPOOL.NS.csv",
                    "donear":"DONEAR.NS.csv",
                    "raymond":"RAYMOND.NS.csv"}

dataFrameMap = dict()
for key, value in stocksToBeLoaded.items():
  dataFrameMap.update({key:spark.read.csv(stocksToBeLoaded[key], inferSchema = True, header = True)})

for key,value in dataFrameMap.items():
  print(key+" : ")
  value.show(1)

HDFC : 
+-------------------+----------+----------+----------+----------+----------+-------+
|               Date|      Open|      High|       Low|     Close| Adj Close| Volume|
+-------------------+----------+----------+----------+----------+----------+-------+
|2019-01-01 00:00:00|177.100006|182.149994|177.000000|180.300003|177.382294|6246700|
+-------------------+----------+----------+----------+----------+----------+-------+
only showing top 1 row

ICICI : 
+-------------------+----------+----------+----------+----------+----------+-------+
|               Date|      Open|      High|       Low|     Close| Adj Close| Volume|
+-------------------+----------+----------+----------+----------+----------+-------+
|2019-01-01 00:00:00|361.500000|364.799988|356.399994|363.750000|361.981659|7759872|
+-------------------+----------+----------+----------+----------+----------+-------+
only showing top 1 row

cipla : 
+-------------------+----------+----------+----------+----------+----------+

In [0]:
  from pyspark.sql.functions import *
  
  for key,value in dataFrameMap.items():
    value = value.withColumn("stockName",lit(key))
    value.createOrReplaceTempView(key+"View")

In [6]:
'''
     * 1. Most Volatile stocks for intra-day in 2019 (descending order)
     * Description :  stock that remained least stable.
     * average of absolute difference between daily high and low price.
     
'''

def createQueryForMaxIntraDayVolatility(stocks):
  query = "";
  for stock in stocks :
    if stock != stocks[0]:
        query += "UNION"
        
    subQuery = "SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent "\
          + " FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio"\
          + " FROM " + stock + "View )"\
          + " GROUP BY stock"
    print(subQuery)
    query += "( " + subQuery + " )"
  query += "ORDER BY intraDayAvgVolatilityPercent DESC"
  return query;

intraDayAvgVolatility = spark.sql(createQueryForMaxIntraDayVolatility(list(stocksToBeLoaded.keys())));
print("The most volatile stocks (intra-day) in 2019 are in order : ");
intraDayAvgVolatility.show(20)

SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio FROM HDFCView ) GROUP BY stock
SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio FROM ICICIView ) GROUP BY stock
SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio FROM ciplaView ) GROUP BY stock
SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio FROM HULView ) GROUP BY stock
SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low)/Open AS dailyDiffRatio FROM sunpharmaView ) GROUP BY stock
SELECT stock, AVG(dailyDiffRatio) AS intraDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(High - Low

In [7]:
'''
     * 2. Most Volatile stocks for inter-day in 2019 (descending order)
     * Description :  stock that remained least stable overnight.
     * average of absolute difference between opening price and closing price of previous day.
     
'''

def createQueryForMaxInterDayVolatility(stocks):
  query = "";
  for stock in stocks:
    if stock != stocks[0]:
      query += "UNION"
    subQuery = "SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent "\
          +" FROM ( SELECT stockName AS stock,"\
          +" 100*abs(Close - (lead(Open) over (order by Date)))/Close AS interDayDiffRatio"\
          +" FROM " + stock + "View )"\
          +" GROUP BY stock"
    print(subQuery)
    query += "( " + subQuery + " )"
  query += "ORDER BY interDayAvgVolatilityPercent DESC"
  return query

interDayAvgVolatility = spark.sql(createQueryForMaxInterDayVolatility(list(stocksToBeLoaded)));
print("The volatile stocks(inter-day) in 2019 are in order : ");
interDayAvgVolatility.show(20)

SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(Close - (lead(Open) over (order by Date)))/Close AS interDayDiffRatio FROM HDFCView ) GROUP BY stock
SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(Close - (lead(Open) over (order by Date)))/Close AS interDayDiffRatio FROM ICICIView ) GROUP BY stock
SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(Close - (lead(Open) over (order by Date)))/Close AS interDayDiffRatio FROM ciplaView ) GROUP BY stock
SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(Close - (lead(Open) over (order by Date)))/Close AS interDayDiffRatio FROM HULView ) GROUP BY stock
SELECT stock, AVG(interdayDiffRatio) AS interDayAvgVolatilityPercent  FROM ( SELECT stockName AS stock, 100*abs(Close - (lead(Open) over (order by 

In [8]:
'''
     * 3. Most profit earned per unit stock in 2019 (descending order)
     * Description :  which stock would have given most profit per unit in year 2019,
     *  assuming it was bought on first day and sold on last day of 2019.
'''

def createQueryForMaxProfit(stocks):
  query = "";
  for stock in stocks:
    if stock != stocks[0]:
      query += "UNION"
    subQuery = "SELECT stock, SUM(price) AS profit, SUM(basePrice) AS basePrice "\
          +" FROM ( SELECT stockName AS stock, Close as price, 0 as basePrice FROM " + stock + "View WHERE Date = '2019-12-31'"\
          +" UNION"\
          +" SELECT stockName AS stock, Open*-1 as price, Open as basePrice  FROM " + stock + "View WHERE Date = '2019-01-01' )"\
          +" GROUP BY stock"
    print(subQuery)
    query += "( " + subQuery + " )"
  query += "ORDER BY profit DESC"
  return query

profits = spark.sql(createQueryForMaxProfit(list(stocksToBeLoaded)));
print("The profit earned per unit stock in 2019 is in order : ");
profits.show()

SELECT stock, SUM(price) AS profit, SUM(basePrice) AS basePrice  FROM ( SELECT stockName AS stock, Close as price, 0 as basePrice FROM HDFCView WHERE Date = '2019-12-31' UNION SELECT stockName AS stock, Open*-1 as price, Open as basePrice  FROM HDFCView WHERE Date = '2019-01-01' ) GROUP BY stock
SELECT stock, SUM(price) AS profit, SUM(basePrice) AS basePrice  FROM ( SELECT stockName AS stock, Close as price, 0 as basePrice FROM ICICIView WHERE Date = '2019-12-31' UNION SELECT stockName AS stock, Open*-1 as price, Open as basePrice  FROM ICICIView WHERE Date = '2019-01-01' ) GROUP BY stock
SELECT stock, SUM(price) AS profit, SUM(basePrice) AS basePrice  FROM ( SELECT stockName AS stock, Close as price, 0 as basePrice FROM ciplaView WHERE Date = '2019-12-31' UNION SELECT stockName AS stock, Open*-1 as price, Open as basePrice  FROM ciplaView WHERE Date = '2019-01-01' ) GROUP BY stock
SELECT stock, SUM(price) AS profit, SUM(basePrice) AS basePrice  FROM ( SELECT stockName AS stock, Close 

In [9]:
'''
    
     * 4. Most profitable stocks in 2019 (descending order)
     * Description :  which stock would have given most profit in year 2019,
     *  assuming it was bought on first day and sold on last day of 2019.
'''


profits.createTempView("profits");

def createQueryforMaxPercentageProfit():
  query = "";
  query = "SELECT profits.stock, profits.profit, profits.basePrice,"\
        +" (100*(profits.profit / profits.basePrice)) AS profitPercent FROM profits "
  query += "ORDER BY profitPercent DESC"
  return query;


profitPercentage = spark.sql(createQueryforMaxPercentageProfit());
print("The % profit earned per unit stock in 2019 is in order : ");
profitPercentage.show(20)


The % profit earned per unit stock in 2019 is in order : 
+----------+-------------------+-----------+-------------------+
|     stock|             profit|  basePrice|      profitPercent|
+----------+-------------------+-----------+-------------------+
| whirlpool|             953.75|    1419.75|  67.17731995069555|
|     ICICI| 177.40002400000003|      361.5|   49.0733123098202|
|  reliance| 388.80004899999994|    1125.25|  34.55232606087535|
|      HDFC| 53.799987999999985| 177.100006| 30.378309529814462|
|       DLF| 53.799987999999985| 177.100006| 30.378309529814462|
| tataMotor| 11.649993999999992|      173.5|  6.714693948126797|
|       HUL|  98.19995100000006|1824.800049|  5.381408831823198|
| sunpharma|0.04998799999998482|      432.5|  0.011557919075141|
|  symphony|              -19.5|1181.949951|-1.6498160504598218|
|     cipla|-41.799987999999985|      520.0| -8.038459230769227|
|      ONGC|-21.199996999999996|      150.0|-14.133331333333333|
|    donear|              -7.25|